# Day 09. Exercise 04
# Pipelines and OOP

## 0. Imports

In [340]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
import joblib
from tqdm.notebook import tqdm
from sklearn.pipeline import Pipeline

## 1. Preprocessing pipeline

Create three custom transformers, the first two out of which will be used within a [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. `FeatureExtractor()` class:
 - Takes a dataframe with `uid`, `labname`, `numTrials`, `timestamp` from the file [`checker_submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
 - Extracts `hour` from `timestamp`.
 - Extracts `weekday` from `timestamp` (numbers).
 - Drops the `timestamp` column.
 - Returns the new dataframe.


2. `MyOneHotEncoder()` class:
 - Takes the dataframe from the result of the previous transformation and the name of the target column.
 - Identifies all the categorical features and transforms them with `OneHotEncoder()`. If the target column is categorical too, then the transformation should not apply to it.
 - Drops the initial categorical features.
 - Returns the dataframe with the features and the series with the target column.


3. `TrainValidationTest()` class:
 - Takes `X` and `y`.
 - Returns `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).


In [341]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        data = X.copy()
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data['hour'] = data['timestamp'].dt.hour
        data['weekday'] = data['timestamp'].dt.weekday
        data = data.drop(columns=['timestamp'])
        return data

In [342]:
class MyOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, target_column):
        self.target_column = target_column
        self.encoder = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
        self.categorial_cols = None
    def fit(self, X, y=None):
        self.categorial_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
        if self.target_column in self.categorial_cols:
            self.categorial_cols.remove(self.target_column)
        if self.categorial_cols:
            self.encoder.fit(X[self.categorial_cols])
        return self
    def transform(self, X):
        data = X.copy()
        y = data[self.target_column]
        
        if self.categorial_cols:
            encode = self.encoder.transform(data[self.categorial_cols])
            new_cols = self.encoder.get_feature_names_out(self.categorial_cols)
            enc_df = pd.DataFrame(encode,X.index, new_cols)
            data = pd.concat([X, enc_df], axis=1)
            y = data.pop(self.target_column)
            data = data.drop(columns = self.categorial_cols)
        return data, y


In [343]:
class TrainValidationTest:
    def __init__(self):
        self.test_size = 0.2
        self.random_state = 21
    def split(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=self.test_size,random_state=self.random_state,stratify=y)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train,y_train,test_size=self.test_size,random_state=self.random_state,stratify=y_train)
        return X_train, X_valid, X_test, y_train, y_valid, y_test
    

## 2. Model selection pipeline

`ModelSelection()` class

 - Takes a list of `GridSearchCV` instances and a dict where the keys are the indexes from that list and the values are the names of the models, the example is below in the reverse order (from high-level to low-level perspective):

```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

 - Method `choose()` takes `X_train`, `y_train`, `X_valid`, `y_valid` and returns the name of the best classifier among all the models on the validation set
 - Method `best_results()` returns a dataframe with the columns `model`, `params`, `valid_score` where the rows are the best models within each class of models.

```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.772727
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.801484
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.855288
```

 - When you iterate through the parameters of a model class, print the name of that class and show the progress using `tqdm.notebook`, in the end of the cycle print the best model of that class.

```
Estimator: SVM
100%
125/125 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
57/57 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
284/284 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

In [344]:
class ModelSelection:
    def __init__(self, grids, grid_dict):
        self.grids = grids
        self.grid_dict = grid_dict
        self.results = []
    def choose(self, X_train, y_train, X_valid, y_valid):
        best_score = 0
        best_model = ""
        for i, grid in enumerate(self.grids):
            model_name = self.grid_dict[i]
            print(f"Estimator {model_name}")
            
            with tqdm(total=len(grid.param_grid)) as pbar:
                grid.fit(X_train,y_train)
                pbar.update(len(grid.param_grid))

            best_params = grid.best_params_
            best_train = grid.best_score_
            valid_score = grid.score(X_valid,y_valid)
            self.results.append({'model': model_name,
                                 'params': best_params,
                                 'valid_score': valid_score})
            print(f"Best params: {best_params}")
            print(f"Best training accuracy: {best_train:.3f}")
            print(f"Validation set accuracy score for best params: {valid_score:.3f}\n")
            if valid_score > best_score:
                best_score = valid_score
                best_model = model_name

        print(f"Classifier with best validation set accuracy: {best_model}")
        return best_model
    def best_results(self):
        res = pd.DataFrame(self.results)
        res = res.sort_values('valid_score', ascending=False).reset_index(drop=True)
        return res
    

## 3. Finalization

`Finalize()` class
 - Takes an estimator.
 - Method `final_score()` takes `X_train`, `y_train`, `X_test`, `y_test` and returns the accuracy of the model as in the example below:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 - Method `save_model()` takes a path, saves the model to this path and prints that the model was successfully saved.

In [356]:
class Finalize:
    def __init__(self, estimator):
        self.estimator = estimator
    def final_score(self, X_train, y_train, X_test, y_test):
        self.estimator.fit(X_train, y_train)
        score = self.estimator.score(X_test,y_test)
        print(f"Accuracy of the final model is: {score}")
        return score
    def save_model(self, path):
        joblib.dump(self.estimator, path)
        print(f"Model saved as: {path}")

## 4. Main program

1. Load the data from the file (****name of file****).
2. Create the preprocessing pipeline that consists of two custom transformers: `FeatureExtractor()` and `MyOneHotEncoder()`:
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Use that pipeline and its method `fit_transform()` on the initial dataset.
```
data = preprocessing.fit_transform(df)
```
4. Get `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` using `TrainValidationTest()` and the result of the pipeline.
5. Create an instance of `ModelSelection()`, use the method `choose()` applying it to the models that you want and parameters that you want, get the dataframe of the best results.
6. create an instance of `Finalize()` with your best model, use method `final_score()` and save the model in the format: `name_of_the_model_{accuracy on test dataset}.sav`.

That is it, congrats!

In [346]:
df = pd.read_csv('../../datasets/checker_submits.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1686 entries, 0 to 1685
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   uid        1686 non-null   object
 1   labname    1686 non-null   object
 2   numTrials  1686 non-null   int64 
 3   timestamp  1686 non-null   object
dtypes: int64(1), object(3)
memory usage: 52.8+ KB


In [347]:
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()),
                          ('onehot_encoder', MyOneHotEncoder(target_column='weekday'))])


In [348]:
data, target = preprocessing.fit_transform(df)

In [349]:
train_val = TrainValidationTest()
X_train, X_valid, X_test, y_train, y_valid, y_test = train_val.split(data, target)

In [350]:
svm_params = {'random_state': [21],
              'probability': [True],
              'kernel': ['linear', 'rbf', 'sigmoid'],
              'C': [0.01, 0.1, 1, 10],
              'gamma': ['scale', 'auto'],
              'class_weight': ['balanced', None]}

tr_params = {'random_state': [21],
             'criterion': ['gini', 'entropy'],
             'max_depth': [None, 10, 20, 30],
             'class_weight': ['balanced', None]}

rf_params = {'random_state': [21],
             'n_estimators': [50, 100],
             'criterion': ['gini', 'entropy'],
             'max_depth': [None, 20, 30],
             'class_weight': ['balanced', None]}
gs_svm = GridSearchCV(estimator=SVC(), param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=-1)
gs_tree = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=tr_params, scoring='accuracy', cv=2, n_jobs=-1)
gs_rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=rf_params, scoring='accuracy', cv=2, n_jobs=-1)


In [351]:
grids = [gs_svm, gs_tree, gs_rf]
grid_dict = {0: 'SVM', 1: 'Decision Tree', 2: 'Random Forest'}

model_selection = ModelSelection(grids, grid_dict)
select_model = model_selection.choose(X_train, y_train, X_valid, y_valid)

best_results_df = model_selection.best_results()
print("Best Results DataFrame:")
print(best_results_df)



Estimator SVM


  0%|          | 0/6 [00:00<?, ?it/s]

Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878

Estimator Decision Tree


  0%|          | 0/4 [00:00<?, ?it/s]

Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': None, 'random_state': 21}
Best training accuracy: 0.807
Validation set accuracy score for best params: 0.863

Estimator Random Forest


  0%|          | 0/5 [00:00<?, ?it/s]

Best params: {'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'n_estimators': 100, 'random_state': 21}
Best training accuracy: 0.853
Validation set accuracy score for best params: 0.896

Classifier with best validation set accuracy: Random Forest
Best Results DataFrame:
           model                                             params  \
0  Random Forest  {'class_weight': None, 'criterion': 'gini', 'm...   
1            SVM  {'C': 10, 'class_weight': None, 'gamma': 'auto...   
2  Decision Tree  {'class_weight': 'balanced', 'criterion': 'gin...   

   valid_score  
0     0.896296  
1     0.877778  
2     0.862963  


In [365]:
if select_model == 'SVM':
    best_model = gs_svm.best_estimator_
elif select_model == 'Decision Tree':
    best_model = gs_tree.best_estimator_
elif select_model == 'Random Forest':
    best_model = gs_rf.best_estimator_

final = Finalize(best_model)
test_accuracy = final.final_score(pd.concat([X_train, X_valid], axis=0), pd.concat([y_train, y_valid], axis=0), X_test, y_test)
model_filename = f"{select_model.replace(' ', '_')}_{test_accuracy:.5f}.sav"
final.save_model(model_filename)


Accuracy of the final model is: 0.9260355029585798
Model saved as: Random_Forest_0.92604.sav


In [366]:
model= joblib.load('Random_Forest_0.90828.sav')
final = Finalize(model)
d = final.final_score(pd.concat([X_train, X_valid], axis=0), pd.concat([y_train, y_valid], axis=0), X_test, y_test)

Accuracy of the final model is: 0.9260355029585798
